In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pprint
import json
import urllib

# Get Covid data from api.covidtracking.com
1. suppose 'data' is the retrieved json, then it has keys ['links', 'meta', 'data']<br>
2. For us/daily, data['data'] stores daily number of cases, with keys ['date', 'states', 'cases', 'testing', 'outcomes']<br>
['date'] is the date in YYYY-MM-DD<br>
['state'] is the number of states<br>
['cases'] has a single key ['total'] with ['value'] (raw number) and ['calculated'] (some statistics)<br>
['testing'] is structured the same way as ['cases']<br>
['outcomes']<br>
3. For state/daily, data['data'] has keys ['date', 'state', 'meta', 'cases', 'tests', 'outcomes']

In [106]:
def get_state_codes():
    l = 'https://api.covidtracking.com/v2/states.json'
    with urllib.request.urlopen(l) as url:
        link_json = json.load(url)
    pd.DataFrame(link_json['data']).to_csv('states.csv')

In [128]:
def retrieve_json(link):
    try:
        with urllib.request.urlopen(link) as url:
            link_json = json.load(url)
        filename = link.split('v2/')[1].replace('/', '_')
        with open(filename, 'w') as f:
            json.dump(link_json, f)
        print('saved ' + filename)
        if filename != 'us_daily.json':
            covid_files.append(filename)
    except:
        print('retrieval failed' + link)

In [113]:
def get_all_json():
    global state_codes
    covidtracking_api = 'https://api.covidtracking.com/v2/'
    
    retrieve_json(covidtracking_api+'us/daily.json')
    
    states_codes = pd.read_csv('states.csv')['state_code'].str.lower()
    for state in states_codes:
        retrieve_json(covidtracking_api + 'states/' + state + '/daily/simple.json')

In [169]:
def clean_df_from_json(filename):
    with open(filename, 'r') as f:
        temp = json.load(f)
    temp = pd.DataFrame(temp['data'])
    
    if filename != 'us_daily.json':
        temp['cases_total'] = temp['cases'].apply(lambda x: x['total'])
        temp['cases_confirmed'] = temp['cases'].apply(lambda x: x['confirmed'])
        temp['cases_probable'] = temp['cases'].apply(lambda x: x['probable'])
        temp['tests_total'] = temp['tests'].apply(lambda x: x['pcr']['specimens']['total'])
        temp['tests_positive'] = temp['tests'].apply(lambda x: x['pcr']['specimens']['positive'])
        temp = temp.drop(['cases', 'tests'], axis=1)
        temp.to_csv(filename.replace('json', 'csv'))
        print('saved ' + filename.replace('json', 'csv'))
    else:
        temp['cases_total'] = temp['cases'].apply(lambda x: x['total']['value'])
        temp['cases_percent'] = temp['cases'].apply(lambda x: x['total']['calculated']['population_percent'])
        temp['cases_increase'] = temp['cases'].apply(lambda x: x['total']['calculated']['change_from_prior_day'])
        temp['cases_7d_change'] = temp['cases'].apply(lambda x: x['total']['calculated']['seven_day_change_percent'])
        temp['testing_total'] = temp['testing'].apply(lambda x: x['total']['value'])
        temp['testing_percent'] = temp['testing'].apply(lambda x: x['total']['calculated']['population_percent'])
        temp['testing_increase'] = temp['testing'].apply(lambda x: x['total']['calculated']['change_from_prior_day'])
        temp['testing_7d_change'] = temp['testing'].apply(lambda x: x['total']['calculated']['seven_day_change_percent'])
        temp = temp.drop(['cases', 'testing'], axis=1)
        temp.to_csv('us_daily.csv')
        print('saved us_daily.csv')

In [170]:
def clean_all_df():
    clean_df_from_json('us_daily.json')
    for file in covid_files:
        clean_df_from_json(file)

In [107]:
get_state_codes()

In [93]:
covid_files = []
get_all_json()

saved states_al_daily_simple.json
saved states_ak_daily_simple.json
saved states_az_daily_simple.json
saved states_ar_daily_simple.json
saved states_ca_daily_simple.json
saved states_co_daily_simple.json
saved states_ct_daily_simple.json
saved states_de_daily_simple.json
saved states_dc_daily_simple.json
saved states_fl_daily_simple.json
saved states_ga_daily_simple.json
saved states_hi_daily_simple.json
saved states_id_daily_simple.json
saved states_il_daily_simple.json
saved states_in_daily_simple.json
saved states_ia_daily_simple.json
saved states_ks_daily_simple.json
saved states_ky_daily_simple.json
saved states_la_daily_simple.json
saved states_me_daily_simple.json
saved states_md_daily_simple.json
saved states_ma_daily_simple.json
saved states_mi_daily_simple.json
saved states_mn_daily_simple.json
saved states_ms_daily_simple.json
saved states_mo_daily_simple.json
saved states_mt_daily_simple.json
saved states_ne_daily_simple.json
saved states_nv_daily_simple.json
saved states_n

In [171]:
clean_all_df()

saved us_daily.csv
saved states_al_daily_simple.csv
saved states_ak_daily_simple.csv
saved states_az_daily_simple.csv
saved states_ar_daily_simple.csv
saved states_ca_daily_simple.csv
saved states_co_daily_simple.csv
saved states_ct_daily_simple.csv
saved states_de_daily_simple.csv
saved states_dc_daily_simple.csv
saved states_fl_daily_simple.csv
saved states_ga_daily_simple.csv
saved states_hi_daily_simple.csv
saved states_id_daily_simple.csv
saved states_il_daily_simple.csv
saved states_in_daily_simple.csv
saved states_ia_daily_simple.csv
saved states_ks_daily_simple.csv
saved states_ky_daily_simple.csv
saved states_la_daily_simple.csv
saved states_me_daily_simple.csv
saved states_md_daily_simple.csv
saved states_ma_daily_simple.csv
saved states_mi_daily_simple.csv
saved states_mn_daily_simple.csv
saved states_ms_daily_simple.csv
saved states_mo_daily_simple.csv
saved states_mt_daily_simple.csv
saved states_ne_daily_simple.csv
saved states_nv_daily_simple.csv
saved states_nh_daily_si

# Get weather data

In [85]:
# use this to get station per state
t = 'https://api.weather.gov/stations?state=CA'
with urllib.request.urlopen(t) as url:
        k = url.read()

In [86]:
j = json.loads(k)['observationStations']

In [87]:
len(j)

3581

In [36]:
with urllib.request.urlopen('https://api.weather.gov/zones/county') as url:
        k2 = url.read()

In [37]:
json.loads(k2).keys()

dict_keys(['@context', 'type', 'features'])

In [64]:
pd.DataFrame(json.loads(k2)['features'][30])

,id,type,geometry,properties
@id,https://api.weather.gov/zones/county/ALC003,Feature,None,https://api.weather.gov/zones/county/ALC003
@type,https://api.weather.gov/zones/county/ALC003,Feature,None,wx:Zone
cwa,https://api.weather.gov/zones/county/ALC003,Feature,None,[MOB]
effectiveDate,https://api.weather.gov/zones/county/ALC003,Feature,None,2020-11-10T18:00:00+00:00
expirationDate,https://api.weather.gov/zones/county/ALC003,Feature,None,2220-11-10T18:00:00+00:00
forecastOffices,https://api.weather.gov/zones/county/ALC003,Feature,None,[https://api.weather.gov/offices/MOB]
id,https://api.weather.gov/zones/county/ALC003,Feature,None,ALC003
name,https://api.weather.gov/zones/county/ALC003,Feature,None,Baldwin
observationStations,https://api.weather.gov/zones/county/ALC003,Feature,None,[]
radarStation,https://api.weather.gov/zones/county/ALC003,Feature,None,None


In [72]:
# Use this to get weather of station
with urllib.request.urlopen('https://api.weather.gov/stations/C1231/observations?limit=10') as url:
        k3 = url.read()

In [73]:
j3 = json.loads(k3)
j3.keys()

dict_keys(['@context', 'type', 'features'])

In [91]:
pd.DataFrame(j3['features'])['properties'][8]

{'@id': 'https://api.weather.gov/stations/C1231/observations/2021-03-09T15:06:00+00:00',
 '@type': 'wx:ObservationStation',
 'elevation': {'value': 12.81, 'unitCode': 'unit:m'},
 'station': 'https://api.weather.gov/stations/C1231',
 'timestamp': '2021-03-09T15:06:00+00:00',
 'rawMessage': '',
 'textDescription': '',
 'icon': None,
 'presentWeather': [],
 'temperature': {'value': 17.77776,
  'unitCode': 'unit:degC',
  'qualityControl': 'qc:V'},
 'dewpoint': {'value': 12.22778,
  'unitCode': 'unit:degC',
  'qualityControl': 'qc:V'},
 'windDirection': {'value': 84,
  'unitCode': 'unit:degree_(angle)',
  'qualityControl': 'qc:V'},
 'windSpeed': {'value': 1.609344,
  'unitCode': 'unit:km_h-1',
  'qualityControl': 'qc:V'},
 'windGust': {'value': 8.04672,
  'unitCode': 'unit:km_h-1',
  'qualityControl': 'qc:S'},
 'barometricPressure': {'value': 103310,
  'unitCode': 'unit:Pa',
  'qualityControl': 'qc:V'},
 'seaLevelPressure': {'value': None,
  'unitCode': 'unit:Pa',
  'qualityControl': 'qc:Z'